### TODO:
- Chart with:
    - Identify pivot points, and build systems that identify resistance and support lines 
    - Identify lines where the price closes and open, plot a line with low alpha and frequently lines will overlay to create a thicker line.

- Make a method to be updating the orderbook every 5 minutes, and do live udpates: https://pythonprogramming.net/python-matplotlib-live-updating-graphs/
    - Update rounding method to not always round down.
    - Add in bollinger band lines and line for current price, and generates graph around current price with $500 bounds
    - Post data to twitter for moolah
- Create a method for viewing the charts by their order id's to see if its the same person generating orders. 
- Figure out how to calculate the number of BTC in circulation at Coinbase, and then use that as a basis to determine how much a price might drop by the change in value of the marketcap (https://www.quora.com/What-is-the-formula-to-calculate-a-price-of-a-cryptocurrency)
- Work on the equity-AEI project
- Get a model to use lagging indicators, and then use all of those values to forecast the price. Need to build the dataset over time. Collected every ten minutes? Need to look at the rate that I can request the full order book.
- I'd want to try and forecast the price how far in the future? Maybe I need a forecasting model, or a timeseries model.

## Readings so far on using ML to estimate price:
- How to do time series analyses in python: https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/
- Should ML be used to forecast prices? : https://machinelearningmastery.com/findings-comparing-classical-and-machine-learning-methods-for-time-series-forecasting/ 
- How to not screw up using ML for forecasting: https://towardsdatascience.com/how-not-to-use-machine-learning-for-time-series-forecasting-avoiding-the-pitfalls-19f9d7adf424

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

In [2]:
import matplotlib.animation as animation

In [3]:
base_url = 'https://api.pro.coinbase.com'

In [4]:
products = '/products'

In [5]:
book = '/book?level='

In [6]:
#Only shows best bid and ask
level_1 = '1'
#Only shows top 50 bids and asks
level_2 = '2'
#Shows all bids and asks
level_3 = '3'

In [7]:
curr1 = 'ETH'
curr2 = 'USD'
pair = '/' + curr1 + '-' + curr2

In [8]:
def round_num(num, round_to_val):
    while num % round_to_val != 0:
        num -= 1
    return num

In [9]:
round_num(111, 5)

110

In [10]:
def gen_ob(pair, level):
    data = requests.get(base_url + products + pair + book + level)
    data = json.loads(data.content.decode('utf-8').replace("'", '"'))
    bids = pd.DataFrame(data['bids'], columns=['price', 'amount', 'id'])
    asks = pd.DataFrame(data['asks'], columns=['price', 'amount', 'id'])
    bids['bid'] = 1
    asks['ask'] = 1
    ob = pd.concat([bids, asks])
    ob = ob.fillna(0)
    ob = ob.sort_values(by='price', ascending=False)
    ob['price'] = ob['price'].apply(lambda price: float(price))
    ob['amount'] = ob['amount'].apply(lambda amnt: float(amnt))
    ob['price'] = np.round(ob['price'], 0)
    ob['price'] = ob['price'].apply(lambda price: round_num(price, 5))
    ob_price = ob.groupby('price')
    result ={
        'price': [],
        'amount': [],
        'bid': [],
        'ask': []
    }
    for price in ob_price:
        result['price'].append(price[0])
        result['amount'].append(np.sum(price[1]['amount']))
        result['bid'].append(price[1]['bid'].tolist()[0])
        result['ask'].append(price[1]['ask'].tolist()[0])
    ob = pd.DataFrame(result)
    ob = ob.sort_values(by='price', ascending=False)
    ob['amount'] = ob['amount'].apply(lambda amnt: float(amnt))
    ob['amount'] = np.round(ob['amount'], 5)
    ob['price'] = ob['price'].apply(lambda price: int(price))
    ob['value'] = ob['price'] * ob['amount']
    curr1 = pair.split('-')[0].strip('/')
    curr2 = pair.split('-')[1]
    ob.to_excel(curr1+'_'+curr2+'_ob.xlsx')
    return ob

In [11]:
og_ob = gen_ob(pair, level_3)

In [12]:
og_ob.head()

,price,amount,bid,ask,value
2319,9999999995,4.00000,0.0,1.0,4.000000e+10
2318,6261919910,0.01000,0.0,1.0,6.261920e+07
2317,1111111180,0.03057,0.0,1.0,3.396667e+07
2316,1000000000,0.40925,0.0,1.0,4.092500e+08
2315,999999995,3.00000,0.0,1.0,3.000000e+09


In [13]:
og_ob.shape

(2320, 5)

In [14]:
fig = plt.figure()

<Figure size 432x288 with 0 Axes>

In [15]:
def ob_range(ob, upper, lower, metric, amount=None, value=None):
    ob = ob[ob.price < upper]
    ob = ob[ob.price > lower]
    ob = ob.sort_values(by='price')
    colors = []
    for index in range(len(ob.price.tolist())):
        if ob['bid'].tolist()[index] == 1:
            colors.append('green')
        if ob['ask'].tolist()[index] == 1:
            colors.append('red')
    if not(amount is None):
        ob = ob[ob.amount > amount]
    if not(value is None):
        ob = ob[ob.value > value]
    plt.clf()
    x = ob['price'].apply(lambda price: str(price)).tolist()
    y = ob[metric].tolist()
    fig.barh(x,y,color=colors)
    fig.title('Bids and Asks', loc='center')
    fig.xticks(rotation=45)
    fig.grid(True)
    fig.show()

In [16]:
def animate(i):
    og_ob = gen_ob(pair, level_3)
    ob_range(og_ob, upper=2500, lower=1800, metric='amount')

In [17]:
fig = plt.figure(figsize=(10,40))
ani = animation.FuncAnimation(fig, animate, interval=10000)

<Figure size 720x2880 with 0 Axes>